# you.com News API <> Llama Index

Creating a chatbot that can provide personalized news on any topic using you.com news API involves several steps, including obtaining the necessary credentials, and programming your chatbot to use the API. 

## Install required packages

In [ ]:
%%capture
! pip install llama-index-retrievers-you==0.1.2
! pip install dotenv==0.0.5

## Load API keys

In [7]:
# assumes a .env file exists with api keys YOU_API_KEY and OPENAI_API_KEY

from dotenv import load_dotenv
import os

load_dotenv()

True

## Create a retriever

In [ ]:
from llama_index.retrievers.you import YouNewsRetriever

you_api_key = os.environ["YOU_API_KEY"]

retriever = YouNewsRetriever(api_key=you_api_key)
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

In [3]:
# TODO until PR is merged, create it manually: https://github.com/run-llama/llama_index/pull/13934

from typing import List, Optional
from llama_index.core.schema import NodeWithScore, TextNode
import requests


class YouNewsRetriever():
    """You retriever."""

    def __init__(
        self,
        api_key: Optional[str] = None
    ) -> None:
        """Init params."""
        self._api_key = api_key or os.environ["YOU_API_KEY"]

    def retrieve(self, query_bundle: str) -> List[NodeWithScore]:
        """Retrieve."""
        headers = {"X-API-Key": self._api_key}
        results = requests.get(
            f"https://api.ydc-index.io/news?q={query_bundle}",
            headers=headers,
        ).json()

        news_hits = [f'{news["description"]}\nURL: {news["url"]}\nage: {news["age"]}' for news in results["news"]["results"]]
        return [NodeWithScore(node=TextNode(text=s), score=1.0) for s in news_hits]

you_api_key = os.environ["YOU_API_KEY"]

retriever = YouNewsRetriever(api_key=you_api_key)
retrieved_results = retriever.retrieve("national parks in the US")

print(retrieved_results[0].get_content())

Our 63 US National Parks across America are a collection of some of the most pristine and beautifully preserved landscapes in the world. From sea to shining sea, nationally protected treasures are filled with stunning views, remarkable wildlife, and historic landmarks.
URL: https://brightstandards.com/us-national-parks-list-map/
age: 5 days ago


# Create a chat engine

Conceptually, it is a stateful analogy of a Query Engine. 

In [4]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Canada")

print(response)

The recent news articles provide a snapshot of various events and developments in Canada. Here are some key highlights:

1. The Bank of Canada cut its key policy rate, becoming the first G7 country to do so in four years. Economists predict further cuts in July.
   Source: Reuters, Yahoo Finance, CNBC

2. Canada Border Services Agency workers are set to strike, potentially causing border chaos.
   Source: CBC News, WCAX

3. Canada demands a portion of revenue from streaming services to support local news and content.
   Source: Ars Technica, Reuters

4. Wall Street anticipates a weaker Canadian dollar and more interest rate cuts following the Bank of Canada's move.
   Source: Bloomberg

5. There are calls to end the practice of "flagpoling" in Canada.
   Source: WCAX

6. Canadian officials are preparing for a meeting with French President Macron, US President Biden, UK's King Charles III, and Canada's PM Trudeau to commemorate D-Day.
   Source: The Guardian

7. Online streaming service

### Include a timeframe

In [5]:
from llama_index.core.chat_engine.context import ContextChatEngine

chat_engine = ContextChatEngine.from_defaults(retriever=retriever)
response = chat_engine.chat("Norway in Jan 2024")

print(response)

In January 2024, Norway experienced some significant events and developments:

1. Norway's $1.6 trillion sovereign wealth fund, the world's largest, reported a record profit of 2.22 trillion crowns ($213 billion) in 2023, driven by strong returns on its investments in technology stocks.

2. January saw combined electric vehicles (EVs) at a 93.9% share in Norway, with a record 92.1% being full electrics (BEVs) and 1.8% plug-in hybrids (PHEVs). This marked a significant increase compared to the previous year.

3. Norway experienced a market crash of -76.6% in January 2023, leading to the lowest market performance in 61 years due to price increases.

4. The country saw a surge in new electric car registrations, with 4,717 new electric cars and 94 plug-in hybrids registered in January 2024.

5. Norway's internet penetration stood at 99.0%, with 5.44 million internet users and 4.49 million social media users in January 2024.

These events showcase Norway's strong focus on sustainability, te

# Create a chat engine with memory

By keeping track of the conversation history, it can answer questions with past context in mind.
 
`condense_plus_context` - A combination of condense_question and context. Look at the chat history and re-write the user message to be a retrieval query for the index. The retrieved text is inserted into the system prompt, so that the chat engine can either respond naturally or use the context from the query engine.

In [33]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.llms.openai import OpenAI

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-3.5-turbo")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    llm=llm,
    verbose=False, # set this to True if you want referenced news articles, like in cell 35
)

In [34]:
# relevant news article with a summary of the news
response = chat_engine.chat("Salesforce")
print(response)

Salesforce, a cloud software giant, recently disappointed investors with its first quarter (FY 2025) earnings report. The company's stock price dropped significantly, experiencing its biggest one-day loss in nearly 20 years after missing revenue consensus. This led to a 20% decline in the stock price. Salesforce CEO Marc Benioff mentioned that the company is cautious about mergers and acquisitions but is open to deals that are seen as beneficial. The poor performance of Salesforce stock has raised concerns among investors and analysts about the company's outlook and adoption of generative AI for software-as-a-service firms like Salesforce.

Various news sources have covered the significant drop in Salesforce's stock price, with articles discussing insider buying and selling, investor worries, and the impact on the Nasdaq. Additionally, there have been mentions of Salesforce being among the worst-performing large-cap stocks in a given week, as well as being identified as the most overso

In [24]:
# uses memory to relate the Hubspot query to market performance news in the query above
response = chat_engine.chat("Hubspot")
print(response)

Condensed question: What is the current situation with Hubspot in the market?
Context: HubSpot shares rose Tuesday after CNBC's David Faber reported that Alphabet is in talks about an all-stock bid for the company.
URL: https://www.cnbc.com/2024/05/28/hubspot-shares-jump-on-talks-of-potential-google-deal-.html
age: 1 week ago

Search engine behemoth, Google is set to take some market share from tech giant MIcrosoft with its latest move to acquire marketing software company Hubspot. According to reports, Google‘s parent compnay Alphabet’s acquisition of the $30 billion-rated company is a component of its ambition ...
URL: https://www.ripplesnigeria.com/google-set-to-acquire-hubspot-in-move-to-take-market-share-from-microsoft/
age: 1 week ago

Learn how to manage email authentication in HubSpot.
URL: https://knowledge.hubspot.com/marketing-email/manage-email-authentication-in-hubspot
age: May 3, 2024

In the most recent trading session, HubSpot (HUBS) closed at $606.09, indicating a +0.2

## Customize news

Using a custom prompt, this news feed can be customized to specific threads, such as this one focused on politics and economics.

In [58]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core import PromptTemplate

custom_prompt = PromptTemplate(
    """\
You're a top-tier reporter focused on news about {question}.\n
YOU SHOULD ALWAYS RESEARCH THE FOLLOWING QUERIES: \n
1. {question} recent news and latest development\n
2. recent regulatory changes affecting {question}\n
3. recent economic events impacting {question}\n
"""
)

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
llm = OpenAI(model="gpt-3.5-turbo")

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    condense_question_prompt=custom_prompt,
    memory=memory,
    llm=llm,
    # verbose=True,
)

In [59]:
response = chat_engine.chat("Mongolia")
print(response)

Mongolia, a country located in East Asia, has been making significant strides in various areas recently. From signing cooperation documents with Belarus to hosting the first international store of Mega Coffee, Mongolia is actively engaging in diplomatic and economic activities. Additionally, Mongolia is focusing on managing its rich uranium market while balancing relationships between the East and West. The country is also implementing reforms to reduce corruption and strengthen its democratic system, as well as establishing a Sovereign Wealth Fund to ensure fair distribution of its resource wealth. Moreover, Mongolia's unique ecosystem, including grasslands that store a substantial amount of terrestrial carbon, is facing challenges like desertification and land degradation. The country's efforts to combat these environmental issues involve incorporating international experiences and local traditions.

Furthermore, Mongolia has been involved in international partnerships, such as Asia 